Zonas Seguras

In [421]:
%pip install tifffile
%pip install opencv-python



Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tifffile in c:\users\user\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages (2024.8.30)



Note: you may need to restart the kernel to use updated packages.


In [422]:
import numpy as np
import cv2
import tifffile
import random


In [423]:
def procesar_entrada(imagenDir=None, n=None, m=None):
    # Si se proporciona una imagen, obtenemos su tamaño automáticamente
    print(imagenDir)
    if imagenDir is not None:

        
        if imagenDir.lower().endswith('.tiff') or imagenDir.lower().endswith('.tif'):
            with tifffile.TiffFile(imagenDir) as tiff:
                num_paginas = len(tiff.pages)
        
                if num_paginas > 1:
                    print("El archivo TIFF es multipágina.")
                    
                    # Leer la primera página
                    heatValue = tiff.pages[0].asarray()
                    size = heatValue.shape
                    print("Primera página leída con tamaño:", heatValue.shape)
                    
                    # Leer las páginas restantes
                    extrasCapas = [tiff.pages[i].asarray() for i in range(1, num_paginas)]
                    print(f"Leídas {len(extrasCapas)} páginas adicionales.")
                else:
                    print("El archivo TIFF es de una sola página.")
                    # Leer la imagen
                    heatValue = tiff.pages[0].asarray()
                    size = heatValue.shape
                    print("Imagen leída con tamaño:", heatValue.shape)
        else:
        # Guardar la imagen como un archivo estándar
            heatValue = cv2.imread(imagenDir)
            if heatValue is not None:
                size = heatValue.shape
                print("Imagen estándar leída con tamaño:", heatValue.shape)
            else:
                print("No se pudo leer el archivo de imagen.")
        # Convertir la imagen a escala de grises para simplicidad
    # Si no se proporciona una imagen, usamos los valores de n y m
    elif n is not None and m is not None:
        size =[n,m]
        # Crear una imagen vacía (negra) de tamaño n x m
        heatValue = np.zeros((n, m),  dtype=np.float32)
        print(f"Imagen de tamaño {n} x {m} creada.")
    
    else:
        raise ValueError("Se debe proporcionar una imagen o los valores n y m.")
    return heatValue, size

In [424]:
def guardarArchivo(nombreDeArchivo,heatValue, direccion =None,multiple =False,extras = None):
    with tifffile.TiffWriter(nombreDeArchivo+".tiff") as tiff:
        tiff.write(heatValue)
        if(multiple and extras != None):
            for i in range(len(extras)):
                tiff.write(extras[i])



In [425]:
def procesoRandom(r,n,m):
    valoresX = list(range(0, m))
    valoresY = list(range(0, n))

    parX = [(a, b) for i, a in enumerate(valoresX) for b in valoresX[i+1:]]
    parY = [(a, b) for i, a in enumerate(valoresY) for b in valoresY[i+1:]]
    
    # Seleccionar n pares aleatorios
    duplasX = random.sample(parX, n)
    duplasY = random.sample(parY, n)

    cords = [[[d1[0],d2[0]],[d1[1],d2[1]]] for d1, d2 in zip(duplasX, duplasY)]

    duplas = random.sample(cords, r)
    
    return duplas

def cubePlay(r,n,m):
    center =[0,0]
    exi = random.randint(0,3)
    if(exi ==0):
        center = [0,random.randint(0,n)]
    elif(exi ==1):
        center = [m,random.randint(0,n)]
    elif(exi ==2):
        center = [random.randint(0,m),0]
    else:
        center = [random.randint(0,m),n]

    
    mel = min(n,m)
    rad = random.randint(mel/8,mel/4)

    
    duplas = []

    for i in range(r):

        
        x1 =center[0]-rad
        x2 =center[0]+rad

        x1 = 0 if x1<0 else x1

        x2 = m if x2>m else x2

        y1 =center[1]-rad
        y2 =center[1]+rad

        y1 = 0 if y1<0 else y1

        y2 = n if y2>n else y2

        duplas.append([[x1,y1],[x2,y2]])

        move = [random.randint(-mel*0.2,mel*0.2),random.randint(-mel*0.2,mel*0.2)]

        move[0] = (-1 if (center[0]+move[0]<0 or center[0]+move[0]>m) else 1) * move[0]
        move[1] = (-1 if (center[1]+move[1]<0 or center[1]+move[1]>n) else 1) * move[1]

        center =[center[0]+move[0],center[1]+move[1]]
        

    return duplas
    


def generarProces(r,n,m,randomm = True):
    if(randomm):
        return procesoRandom(r,n,m)
    else:
        return cubePlay(r,n,m)
    

def GuardarCords(cordss,name):
    with open(name+'.txt', 'w') as f:
        for coord in cordss:
            f.write(f"{coord}\n")

    print("Coordenadas guardadas en '"+name+"'.txt'.")

def LeerCoordenadas(name):
    coordenadas_leidas = []
    with open(name+'.txt', 'r') as f:
        for line in f:
            coordenadas_leidas.append(eval(line.strip()))

    return coordenadas_leidas 

In [426]:
name = "coordenadas"
GuardarCords(generarProces(100,1280,720,False),name)

Coordenadas guardadas en 'coordenadas'.txt'.


In [427]:
def normalizar_matriz(matriz):
    #convierte la matriz a un rango de 0 a 1
    min_val = 0
    max_val = np.max(matriz)
    if max_val - min_val == 0:
        return matriz  # Evitar división por cero si todos los valores son iguales
    return (matriz - min_val) / (max_val - min_val)

def CalcularNuevaDesviacion(preHeat, newHeat): ## formula para calcular el efecto del video sobre los datos previos,
    
    diferencia = preHeat - newHeat
    magnitud_diferencia = np.abs(diferencia)

    direccion = np.where(magnitud_diferencia == 0, 0, diferencia / magnitud_diferencia)  # saber la direccion en la que nos desviamos si el valor es el mismo no debe haber cambio por ende es 0
    
    producto = preHeat * newHeat

    # Calcular el nuevo valor
    newnewHeat = preHeat + direccion * producto

    return newnewHeat

def CalcularNuevaDesviacionAlfa(preHeat, newHeat,alpha): ## formula para calcular el efecto del video sobre los datos previos, utilizando un valor alfa de peso
    # Calcular con el facvotr de suavisado
    newnewHeat = alpha*preHeat + (1-alpha) * newHeat
    return newnewHeat

In [428]:
def procesarCamino(heatValue,size,newPath,calor=False, alpha = 1):
    
    newHeat = np.zeros(heatValue.shape,  dtype=np.float32)
    
    for (x1, y1), (x2, y2) in newPath:
        newHeat[x1:x2+1, y1:y2+1] += 1
    # Normalizar la matriz a un rango de 0 a 255
    print(newHeat[300,200])
    newHeat = normalizar_matriz(newHeat)
    heatValue = normalizar_matriz(heatValue)
    print(newHeat[300,200])
    if(calor):
        newHeat = CalcularNuevaDesviacionAlfa(heatValue,newHeat,alpha)
    else:
        newHeat = CalcularNuevaDesviacion(heatValue,newHeat)
        
    return newHeat

In [429]:
def dividirPorFases(mapa, division, porcent=True):
    percent = 1
    raz = 1
    filas,columnas = mapa.shape

    if(not porcent):
        raz = int(division)
        percent = 1/division 
    else:
        raz = int(1/division)
        percent = division 
    extra = []

    for k in range(raz):
        print(percent*k*255)
        copy  = np.zeros(mapa.shape,dtype=np.uint8)
        for i in range(filas):
            for j in range(columnas):
                if(mapa[i,j]>percent*k):
                    copy[i,j] =255
        extra.append(copy)

    return extra





In [430]:
def procesamientoCaminos(name,n=720,m=1280,calor=True,alpha=0.75,divisions=5):
    heatValue,size = procesar_entrada(name+".tiff", n=n,m=m)
    newPath = LeerCoordenadas("coordenadas")
    # Dimensiones de la imagen (n: alto, m: ancho
    newheatValue = procesarCamino(heatValue,size,newPath,calor,alpha)
    imagen_uint8 = (newheatValue * 255).astype(np.uint8)

    # Mostrar la imagen en una ventana
    cv2.imshow('Imagen Negra', imagen_uint8)

    cv2.setWindowProperty('Imagen Negra', cv2.WND_PROP_TOPMOST, 1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    extrasCapas = dividirPorFases(newheatValue,division=5,porcent=False)

    guardarArchivo(name,newheatValue,multiple=True,extras=extrasCapas)

In [431]:

##variables Globales

heatValue,size = procesar_entrada(n=720,m=1280)

extrasCapas = None

difProcess = 0.0

newPath = LeerCoordenadas("coordenadas")

analize = False


# Dimensiones de la imagen (n: alto, m: ancho
newheatValue = procesarCamino(heatValue,size,newPath,True,0)
imagen_uint8 = (newheatValue * 255).astype(np.uint8)


print(imagen_uint8[300,200])

# Mostrar la imagen en una ventana
cv2.imshow('Imagen Negra', imagen_uint8)

cv2.setWindowProperty('Imagen Negra', cv2.WND_PROP_TOPMOST, 1)
cv2.waitKey(0)
cv2.destroyAllWindows()


None
Imagen de tamaño 720 x 1280 creada.
8.0
0.15384616
39


In [432]:
"""print("diga el porcentaje de separacion") ## tambien se puede con numero de divisiones

percent = input("entre 0 y 1 o 0 y 100 ")
percent = float(percent)


if(percent >1):
    percent = percent/100
"""
print(newheatValue.max())
extrasCapas = dividirPorFases(newheatValue,0.2)



guardarArchivo('mapaDeCalor',newheatValue,multiple=True,extras=extrasCapas)

1.0
0.0
51.0
102.0
153.00000000000003
204.0


In [436]:
name = "coordenadas"
GuardarCords(generarProces(100,1280,720,False),name)
procesamientoCaminos('mapaDeCalor',calor=False)



Coordenadas guardadas en 'coordenadas'.txt'.
mapaDeCalor.tiff
El archivo TIFF es multipágina.
Primera página leída con tamaño: (720, 1280)
Leídas 5 páginas adicionales.
7.0
0.4117647


C:\Users\User\AppData\Local\Temp\ipykernel_18912\2566342434.py:14: RuntimeWarning: invalid value encountered in divide
  direccion = np.where(magnitud_diferencia == 0, 0, diferencia / magnitud_diferencia)  # saber la direccion en la que nos desviamos si el valor es el mismo no debe haber cambio por ende es 0


0.0
51.0
102.0
153.00000000000003
204.0
